In [13]:
import sys
import gc
import joblib
sys.path.append('../../')

from scoring.event_detection_matrix import competition_score
from models.rnn import LightningModel

from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [14]:
checkpoint_path = "../../models/wandb/checkpoints/rnn_1_1024/model.ckpt"
model = LightningModel.load_from_checkpoint(checkpoint_path)

c:\ProgramData\Anaconda3\envs\aich\lib\site-packages\lightning\pytorch\utilities\migration\utils.py:55: The loaded checkpoint was produced with Lightning v2.1.1, which is newer than your current Lightning version: v2.1.0


In [15]:
df_validation_events = pd.read_csv('../../data/processed/validation_events_split.csv')

In [16]:
LAGS_FUTURE = [f"t_lag_{i}" for i in range(-1, -25, -1)]
LAGS_PAST = reversed([f"t_lag_{i}" for i in range(1, 25)])
FEATURES = [*LAGS_PAST, 't_0', *LAGS_FUTURE]

In [17]:
smoothing_length = 12 * 30 # 30 Minutes

def event_extraction(series):
    events = []

    series_id = series["series_id"].values[0]
               
    series["score"] = series["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
    series["smooth"] = series["prediction_confidence_0"].rolling(smoothing_length, center=True).mean().bfill().ffill()

    # Binarize the smoothing column
    series["smooth"] = series["smooth"].round()

    # Getting predicted onset and wakeup time steps
    pred_onsets = series[series['smooth'].diff() > 0]['step'].tolist()
    pred_wakeups = series[series['smooth'].diff() < 0]['step'].tolist()
     
    if len(pred_onsets) > 0 and len(pred_wakeups) > 0:
        print(pred_wakeups)
        print(pred_onsets)

    # Ensuring all predicted sleep periods begin and end
        if min(pred_wakeups) < min(pred_onsets):
            pred_wakeups = pred_wakeups[1:]

        if len(pred_wakeups) > 0 and max(pred_onsets) > max(pred_wakeups) :
            pred_onsets = pred_onsets[:-1]

        if len(pred_wakeups) == 0 or len(pred_onsets) == 0:
            return pd.DataFrame(events)

        # Keeping sleep periods longer than 30 minutes
        sleep_periods = [(onset, wakeup) for onset, wakeup in zip(pred_onsets, pred_wakeups) if wakeup - onset >= 12 * 30]

        for onset, wakeup in sleep_periods :
            # Scoring using mean probability over period
            score = series[(series['step'] >= onset) & (series['step'] <= wakeup)]['score'].mean()

            # Adding sleep event to dataframe
            onset_row = {'row_id': len(events), 'series_id': series_id, 'step': onset, 'event': 'onset', 'score': score}                
            events.append(onset_row)

            wakeup_row = {'row_id': len(events), 'series_id': series_id, 'step': wakeup, 'event': 'wakeup', 'score': score}
            events.append(wakeup_row)

    return pd.DataFrame(events)

In [43]:
def prediction_batch(batch):
    X = batch
    
    with torch.no_grad():
        logits = model(X)
    
    label = torch.argmax(logits, dim=-1)
    confidence = torch.softmax(logits, dim=-1)
    confidence_0 = confidence[:, 0]
    confidence_1 = confidence[:, 1]
    return label, confidence_0, confidence_1

In [44]:
def prediction(overview_info, series):
    predictions = overview_info[['series_id', 'step']]
    
    label_list = []
    confidence_0_list = []
    confidence_1_list = []
    
    # series_length, series_columns = series[FEATURES].values.shape

    dataset = TensorDataset(series)
    dataloader = DataLoader(dataset, batch_size=1024)
    for index, batch in enumerate(dataloader): 
        label, confidence_0, confidence_1 = prediction_batch(batch[0])
        
        label_list.append(label)
        confidence_0_list.append(confidence_0)
        confidence_1_list.append(confidence_1) 
        
    predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()
    predictions['prediction_confidence_0'] = torch.cat(confidence_0_list).cpu().numpy()
    predictions['prediction_confidence_1'] = torch.cat(confidence_1_list).cpu().numpy()

    return predictions

In [45]:
overview = pd.read_parquet('../../data/processed/transformer/validation/overview.parquet')
print(overview.head())
events_list = []

for i, series_id in enumerate(overview.series_id.unique()):
    print(series_id)
    current_series = overview[overview['series_id'] == series_id]
    # print(f'Step {i+1} of {len(series_ids)}')
    series_prepared = torch.load('../../data/processed/transformer/validation/' + series_id + '.pt')
    predictions = prediction(current_series, series_prepared)
    events = event_extraction(predictions)

    events_list.append(events)

    del series_prepared
    del predictions
    gc.collect()

events = pd.concat(events_list).reset_index(drop=True)

      series_id  step  awake  series_index
0  062dbd4c95e6     0      1             0
1  062dbd4c95e6     1      1             1
2  062dbd4c95e6     2      1             2
3  062dbd4c95e6     3      1             3
4  062dbd4c95e6     4      1             4
062dbd4c95e6


C:\Users\oliko\AppData\Local\Temp\ipykernel_12972\3557321736.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()


[994, 1744, 3232, 3508, 4799, 5970, 7087, 16435, 17308, 18460, 18804, 19833, 20125, 21302, 22646, 23661, 24337, 35690, 35836, 51485, 51871, 53976, 55101, 56258, 57961, 70287, 71327, 71592, 71812, 72359, 74169, 74809, 76678, 85953, 87452, 89565, 91775, 93078, 93592, 102792, 102872, 103051, 106758, 108606, 108677, 121252, 121435, 121871, 124489, 124597, 139156, 141268, 141950, 143491, 145325, 145742, 154374, 155231, 157612, 158149, 159390, 160911, 161660, 162391, 162665, 170685, 172905, 173526, 174036, 175042, 175314, 175424, 178569, 179014, 179656, 180922, 190697, 191373, 192710, 196236, 205962, 208264, 209690, 210708, 212088, 224660, 225820, 226391, 227307, 229217, 242941, 244245, 245296, 245625, 246385, 248355, 260586, 260701, 261740, 262042, 262235, 274831, 277450, 277978, 279372, 281057, 291757, 292547, 293103, 294183, 294935, 295130, 297750, 311060, 312220, 312791, 313707, 315617, 326480, 348486, 349446, 349936, 360098, 360993, 362641, 364311, 364387, 365891, 367469, 376732, 377424

C:\Users\oliko\AppData\Local\Temp\ipykernel_12972\3557321736.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()


[2208, 4576, 5116, 15240, 17808, 18672, 19175, 30396, 31348, 33587, 34463, 35629, 45895, 46767, 48278, 50682, 52033, 53429, 55062, 63162, 65258, 68222, 69771, 70983, 71582, 73357, 80639, 81322, 84403, 84677, 86079, 88051, 89624, 102041, 104230, 108103, 115343, 119485, 120991, 136765, 138271, 149903, 154045, 155551, 167183, 171325, 172831, 189414, 190662, 190930, 191180, 191910, 193475, 194058, 201293, 201973, 204690, 206126, 207242, 208471, 209461, 211113, 211653, 218927, 219605, 222924, 223137, 224653, 225417, 227453, 236303, 240445, 241951, 253583, 255817, 256228, 256462, 260622, 270863, 275005, 276511, 288143, 292285, 293791, 305423, 309565, 311071, 322703, 326845, 328351, 339983, 344125, 345631, 357263, 361405, 362911]
[2073, 4221, 5050, 5423, 15732, 18138, 18925, 19501, 30635, 31680, 34182, 35288, 37715, 46601, 46983, 48555, 51383, 53199, 54386, 56678, 63652, 65831, 68543, 70462, 71499, 73153, 73516, 81164, 81335, 84615, 85817, 87234, 89478, 89733, 103199, 107562, 108267, 115984, 

...

[727, 1224, 2169, 3199, 4377, 12331, 12907, 16366, 16450, 16605, 17178, 18491, 21099, 21541, 22744, 29542, 31064, 32051, 35588, 37453, 37882, 40655, 46881, 47638, 49979, 54097, 55234, 55736, 56072, 64738, 65303, 67452, 68372, 69081, 70119, 70352, 71130, 73678, 81539, 85693, 89349, 89998, 90696, 91629, 98436, 99337, 101475, 102687, 102913, 103541, 105985, 106387, 107473, 108163, 108703, 115757, 116417, 118290, 122748, 123491, 125710, 125844, 125846, 133122, 134732, 135163, 135576, 136203, 139951, 141333, 141541, 142212, 142401, 143444, 150445, 151392, 153535, 155372, 155651, 156095, 159182, 160036, 161149, 167856, 168677, 169687, 171730, 174125, 174822, 176555, 177334, 184906, 185658, 186508, 187889, 188367, 189651, 189840, 193633, 194316, 202224, 203533, 203797, 204126, 204384, 208664, 210334, 219884, 222217, 223472, 223900, 224527, 225945, 226799, 229659, 230108, 236693, 237561, 242098, 243076, 245241, 246786, 253873, 256624, 256828, 257389, 258893, 259546, 261999, 263244, 271452, 273

C:\Users\oliko\AppData\Local\Temp\ipykernel_12972\3557321736.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()


[1549, 3286, 4449, 5245, 7107, 15877, 17038, 20947, 22482, 23196, 23619, 23695, 32800, 34599, 35797, 37558, 38166, 38266, 39814, 40880, 49406, 54393, 55509, 56950, 57502, 57973, 68587, 69434, 71517, 71734, 71819, 72044, 72223, 72370, 72724, 72799, 74826, 83962, 86083, 86456, 87812, 89074, 89216, 91442, 92017, 92720, 101750, 104974, 105563, 109615, 109877, 110000, 118905, 121250, 122920, 124600, 126162, 126876, 127157, 127280, 136338, 137990, 140275, 141036, 144748, 146138, 155046, 156350, 161164, 161840, 169956, 172803, 172945, 173325, 174286, 174807, 175870, 177985, 178824, 188178, 189803, 193747, 195282, 195996, 196277, 196400, 205458, 207083, 211027, 212562, 213276, 213557, 213680, 222738, 224363, 228307, 229842, 230556, 230837, 230960, 240018, 241643, 245587, 247122, 247836, 248117, 248240, 257298, 258923, 262867, 264402, 265116, 265397, 265520, 274578, 276203, 280147, 281682, 282396, 282677, 282800, 291858, 293483, 297427, 298962, 299676, 299957, 300080, 309138, 310763, 314707, 31

C:\Users\oliko\AppData\Local\Temp\ipykernel_12972\3557321736.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()


[326, 1726, 3518, 5105, 12976, 19727, 20599, 21057, 21847, 22586, 30461, 36563, 36750, 38130, 38909, 40809, 47618, 54328, 56238, 58177, 65856, 65965, 66672, 69892, 72863, 73479, 73969, 74184, 74389, 74584, 83356, 84639, 87127, 87788, 89218, 89250, 89641, 90238, 91389, 101158, 102531, 102805, 106768, 107969, 108521, 116641, 117087, 117183, 123914, 125002, 126661, 133915, 134319, 135681, 143690, 144057, 144235, 151701, 152577, 153554, 153864, 154057, 155280, 156409, 157800, 158051, 158615, 160480, 161263, 169083, 175267, 176470, 177088, 177722, 179451, 179943, 186937, 187636, 189142, 189476, 190160, 191045, 192848, 194679, 196985, 204251, 213090, 220660, 220855, 225617, 226128, 227019, 227644, 228368, 229046, 229619, 238127, 244118, 244178, 245224, 245808, 247626, 255334, 262764, 264774, 272485, 278440, 279469, 281930, 289451, 290189, 296426, 297013, 297792, 298943, 299526, 300428, 305792, 306957, 307066, 307346, 307903, 308855, 312358, 314131, 316362, 317395, 324532, 326421, 331444, 335

In [46]:
competition_score(df_validation_events, events)

0.0026386691183066923

In [47]:
events.to_csv('./data/predictions_1024.csv', index=False)